<a href="https://colab.research.google.com/github/belalabouzaid/siads699_team13_collab/blob/main/Notebooks/5a-XGB_Model_Tuning_DASK_with_Kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Notebook Objectives
The aim of this notebook is to tune the XGB regression model by optimyzing the hyperparameters. The model tuning will be done by means of using K-fold Gridsearch CV. As this is a computationally expensive step, we have used DASK to access additional computational resources.

In [ ]:
# Import libraries
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Import libraries
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# To work with numpy arrays
import numpy as np
# ML model building, training and testing
import sklearn
# Kfold split
from sklearn.model_selection import KFold
# Build ML pipeline
from sklearn.pipeline import Pipeline
# Liner regression model building
from sklearn.linear_model import LinearRegression
# Polynomial regression
from sklearn.preprocessing import PolynomialFeatures
# sklearn included transformer
from sklearn.preprocessing import QuantileTransformer
# to split data into test and train
from sklearn.model_selection import train_test_split
# Scaling transform for PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# To run PCA feature reduction
from sklearn.decomposition import PCA
# XGBoost model
from xgboost import XGBRegressor,XGBClassifier
# Random forest regressor
from sklearn.ensemble import RandomForestRegressor
# for regression error calculation
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import learning_curve
# for regression r2 score calculation
from sklearn.metrics import r2_score
# for regression mse score calculation
from sklearn.metrics import mean_squared_error
# For hyperparameter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
# To transform the target variable into a normal distribution
from sklearn.compose import TransformedTargetRegressor
# visualisation
from matplotlib import pyplot

# Gridsearch (XGB hyperparameter tuning) using DASK

In [ ]:
pip install "dask[complete]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.5 MB/s eta 0:00:00


In [ ]:
pip install dask-ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 9.5 MB/s eta 0:00:00


In [ ]:
import joblib
from dask.distributed import Client

In [ ]:
# Instantiate pipeline
pipe = Pipeline([
               # ('standard_scaler', StandardScaler())
              #, ('PCA', PCA(n_components= 6)) ,
              ('xgb', XGBRegressor(random_state= 42))
              ])

In [ ]:
param_grid = {
            #'PCA__n_components':[6],
            'xgb__max_depth': [5,10,15,20],
            'xgb__colsample_bylevel': [0.5,0.7,0.8],
            'xgb__learning_rate': [0.001,0.01,0.1],
            'xgb__n_estimators': [100]
            }

In [ ]:
import dask_ml.model_selection as dcv

In [ ]:
client = Client() # start a local Dask client

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:39185
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42333'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:44731'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:40265', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:40265
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:42626
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:35531', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:35531
INFO:distributed.core:Sta

## Model Tuning using Three Rivers 22 data

In [ ]:
threerivers_22 = pd.read_csv('https://github.com/belalabouzaid/siads699_team13_collab/raw/main/Data/threerivers_2022.csv')

In [ ]:
X = threerivers_22.drop(columns = 'smi')
y = threerivers_22['smi']

In [ ]:
X = X[['aspect','slope','NDMI','SR_B5','MSI','SAVI','EVI','NDVI','ST_B10'
                                ,'MNDWI','SR_B7','SR_B6','MSAVI']]

In [ ]:
#Splitting the data into test (25%) and train (75%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
%%time
# Quick test K = 2
with joblib.parallel_backend('dask'):
  grid = dcv.GridSearchCV(pipe, param_grid, cv = 2, n_jobs = -1, scoring = 'r2')
  grid.fit(X_train, y_train)

CPU times: user 30.4 s, sys: 3.91 s, total: 34.3 s
Wall time: 7min 33s


In [ ]:
grid.best_params_

{'xgb__colsample_bylevel': 0.8,
 'xgb__learning_rate': 0.1,
 'xgb__max_depth': 10,
 'xgb__n_estimators': 100}

In [ ]:
grid.best_score_

0.9815606027732697

In [ ]:
%%time
# K = 10
with joblib.parallel_backend('dask'):
  grid = dcv.GridSearchCV(pipe, param_grid, cv = 10, n_jobs = -1, scoring = 'r2')
  grid.fit(X_train, y_train)

CPU times: user 4min 21s, sys: 29 s, total: 4min 50s
Wall time: 56min 34s


In [ ]:
grid.best_params_

{'xgb__colsample_bylevel': 0.7,
 'xgb__learning_rate': 0.1,
 'xgb__max_depth': 10,
 'xgb__n_estimators': 100}

In [ ]:
grid.best_score_

0.9828189286623382

## Model Tuning using Mariposa 21 data

In [ ]:
mariposa_21 = pd.read_csv('https://github.com/belalabouzaid/siads699_team13_collab/raw/main/Data/mariposa_2021.csv')

In [ ]:
X = mariposa_21.drop('smi',1)
y = mariposa_21['smi']

<ipython-input-19-4d28256c0d64>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = mariposa_21.drop('smi',1)


In [ ]:
X = X[['aspect','slope','NDMI','SR_B5','MSI','SAVI','EVI','NDVI','ST_B10'
                                ,'MNDWI','SR_B7','SR_B6','MSAVI']]

In [ ]:
#Splitting the data into test (25%) and train (75%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
%%time
# Quick test K = 2
with joblib.parallel_backend('dask'):
  grid = dcv.GridSearchCV(pipe, param_grid, cv = 2, n_jobs = -1, scoring = 'r2')
  grid.fit(X_train, y_train)

CPU times: user 1min 7s, sys: 9.02 s, total: 1min 16s
Wall time: 17min 20s


In [ ]:
grid.best_params_

{'xgb__colsample_bylevel': 0.8,
 'xgb__learning_rate': 0.1,
 'xgb__max_depth': 10,
 'xgb__n_estimators': 100}

In [ ]:
grid.best_score_

0.8245801889809902

In [ ]:
%%time
# K = 10
with joblib.parallel_backend('dask'):
  grid = dcv.GridSearchCV(pipe, param_grid, cv = 10, n_jobs = -1, scoring = 'r2')
  grid.fit(X_train, y_train)

CPU times: user 9min 1s, sys: 1min 2s, total: 10min 3s
Wall time: 2h 2min 9s


In [ ]:
grid.best_params_

{'xgb__colsample_bylevel': 0.7,
 'xgb__learning_rate': 0.1,
 'xgb__max_depth': 10,
 'xgb__n_estimators': 100}

In [ ]:
grid.best_score_

0.8343250412675868